Bootstrapping looks simple on the surface: re-sample and compute standard errors. But while the tool is robust, it still requires some care.

In particular, it is very important to mimic the original experiment in every way, down to the last detail, in order to compute the bootstrapped standard errors correctly. In this short example, we demonstrate how recognizing a small subtlety in the experiment leads to different standard errors and ultimately a different conclusion.

First, we begin by creating some simulated data.

In [1]:
# Load libraries
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm

# Set a random seed
np.random.seed(4)

# Initialize data with fifteen treated and ten control units
data = pd.concat(
    [pd.DataFrame(np.ones(15), columns = ['treatment']), 
     pd.DataFrame(np.zeros(10), columns = ['treatment'])])

# Create outcome variable, as function of treatment effect and noise
beta = 2.27
data['outcome'] = data['treatment'] * beta + np.random.normal(0, 2, 25)

What does the data look like?

In [2]:
print(data)

    treatment   outcome
0         1.0  2.371123
1         1.0  3.269903
2         1.0  0.278182
3         1.0  3.657197
4         1.0  1.433397
5         1.0 -0.899154
6         1.0  0.974586
7         1.0  3.467150
8         1.0  2.934500
9         1.0 -0.024953
10        1.0  3.507339
11        1.0  2.094026
12        1.0  3.120145
13        1.0  2.934506
14        1.0 -0.043633
0         0.0  0.701994
1         0.0 -1.213775
2         0.0  3.093959
3         0.0  1.446683
4         0.0  0.092271
5         0.0 -1.965983
6         0.0  0.108865
7         0.0  0.319786
8         0.0 -2.417896
9         0.0  4.446720


Now we remind ourselves of the regression function, and what it does, and how we can view and programmatically extract the results.

In [3]:
# Conduct linear regression
result = sm.ols(formula = 'outcome ~ treatment', data = data).fit()
# print(result)
# print(result.summary())
# print(result.params)
print(result.params[1])

1.4770252040856577


With that, we more generally embed the regression tool in a custom function that returns just the coefficient from regression.

In [4]:
# Define a regression function
def regression(f, df):
  result = sm.ols(formula = f, data = df).fit()
  return(result.params[1])

# Get point estimate
point_estimate = regression('outcome ~ treatment', data)
print("Estimated effect of treatment is: " + str(round(point_estimate, 3)))

# Get summary on number of treated / control units
print("There are " + str(sum(data['treatment'])) + " treated units and " + 
      str(len(data['treatment']) - sum(data['treatment'])) + " control units")

Estimated effect of treatment is: 1.477
There are 15.0 treated units and 10.0 control units


Before we move on, take a moment to think about the different ways we might compute the bootstrapped standard errors.

Be specific about the re-sampling design that you might implement, and why you made the design choices that you did.

In doing so, notice that there are ten control units, and fifteen treated units. There are a few plausible hypotheses:


1.   There are supposed to be exactly ten control units and exactly fifteen treated units.
2.   Each unit is allocated with 60% probability to treatment.
3.   Each unit is allocated with 50% probability to treatment. Due to some random chance, there are more treated than control units.

Notice how the setup of the bootstrap computation differs for each one, and how the results change.

Consider the first scheme, in which there are exactly ten control units and fifteen treated units in the original experiment design, and this is mimicked in the bootstrap.

In [5]:
# Implement bootstrap under Scheme 1: 
# Exactly ten control units and fifteen treated units

# Identify row numbers for treated and control units
index_treated = np.where(data['treatment'] == 1)[0]
index_control = np.where(data['treatment'] == 0)[0]
bootstrap = []

# For each bootstrap iteration, sample treated and control units separately,
# thus preserving 15 treated and 10 control units in every iteration
for i in range(2000):
  treated_sample = np.random.choice(index_treated, len(index_treated), 
                                    replace = True)
  control_sample = np.random.choice(index_control, len(index_control), 
                                    replace = True)
  sample = np.append(treated_sample, control_sample)
  estimate = regression('outcome ~ treatment', data.iloc[sample,])
  bootstrap.append(estimate)

# Compute standard deviation of the point estimates across all iterations, 
# which is the standard error of the original point estimate
print("The standard error of the estimate: " + str(round(np.std(bootstrap), 4)))
print("The t-statistic is: " + str(round(point_estimate/np.std(bootstrap), 4)))

The standard error of the estimate: 0.7297
The t-statistic is: 2.0242


In [ ]:
#sorted(bootstrap)

Consider the second scheme, in which each unit is allocated to treatment with 60% probability, and this is mimicked in the bootstrap.

In [8]:
# Implement bootstrap under Scheme 2: 
# Any unit has a 60% probabability of being assignment to treatment
bootstrap = []

# For each bootstrap iteration, sample among the observations with equal 
# weights, so that our samples are 60% treated on average
for i in range(2000):
  sample = np.random.choice(range(25), 25, replace = True)
  bootstrap.append(regression('outcome ~ treatment', data.iloc[sample,]))

# Compute standard deviation of the point estimates across all iterations, 
# which is the standard error of the original point estimate
print("The standard error of the estimate: " + str(round(np.std(bootstrap), 4)))
print("The t-statistic is: " + str(round(point_estimate/np.std(bootstrap), 4)))

The standard error of the estimate: 0.7558
The t-statistic is: 1.9542


Consider the third scheme, in which each unit is allocated to treatment with 50% probability, and this is mimicked in the bootstrap.

In [10]:
# Implement bootstrap under Scheme 3: 
# Any unit has a 50% probabability of being assignment to treatment (and it 
# just so happens that our sample is unevenly split)

# For each bootstrap iteration, sample among the observations with unequal 
# weights, so that our samples are 50% treated on average

# Thus, we first have to construct the new weights
weight_control = sum(data['treatment'] == 1)/len(data['treatment'])
weight_treatment = 1 - weight_control
probability = ((data['treatment'] == 1) * weight_treatment + 
               (data['treatment'] == 0) * weight_control)
probability = probability/sum(probability)
print(probability)
print(weight_control)
print(weight_treatment)

0     0.033333
1     0.033333
2     0.033333
3     0.033333
4     0.033333
5     0.033333
6     0.033333
7     0.033333
8     0.033333
9     0.033333
10    0.033333
11    0.033333
12    0.033333
13    0.033333
14    0.033333
0     0.050000
1     0.050000
2     0.050000
3     0.050000
4     0.050000
5     0.050000
6     0.050000
7     0.050000
8     0.050000
9     0.050000
Name: treatment, dtype: float64
0.6
0.4


In [12]:
# Now conduct the bootstrap
bootstrap = []

for i in range(2000):
  sample = np.random.choice(range(25), 25, replace = True, p = probability)
  bootstrap.append(regression('outcome ~ treatment', data.iloc[sample,]))

# Compute standard deviation of the point estimates across all iterations, 
# which is the standard error of the original point estimate
print("The standard error of the estimate: " + str(round(np.std(bootstrap), 4)))
print("The t-statistic is: " + str(round(point_estimate/np.std(bootstrap), 4)))

The standard error of the estimate: 0.7391
The t-statistic is: 1.9984


As a discussion point: why are the standard errors largest for the second approach, and smallest for the first and third approach?